# Calculate the average NDI over a sample area through time 
**Option (c): Time-series mean seasonal NDI**

## Log in to Google Earth Engine

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.

drive.mount('/content/drive')  # Mount Google Drive onto Colab

## Identify region and NDI of interest

In [ ]:
# Location - editable
latitude = 41.017           # Degrees of latitude
longitude = -123.611        # Degrees of longitude
location_name = 'hoopa'     # recognisable name, to create a useful file name
sample_radius = 1000        # radius of sample area around chosen point, in metres

# Date range - editable
year_begin = 2014
year_end = 2023
month_begin = 8
month_end = 10

# NDI - editable
type_of_ndi = 'NDVI'  # type of NDI calculated, for filename purposes
r_high = 'B5'  # relevant band name for r_high
r_low = 'B4'   # relevant band name for r_low

# You can edit this variable
folder = 'scires_project_2C'



Sanity check the sample region

In [ ]:
# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
sample_region = point.buffer(sample_radius)     # Buffer the point

Map = geemap.Map()  # Create an empty Map
Map.addLayer(sample_region, {}, "Search Region")  # Add our AOI
Map.centerObject(sample_region, zoom=12)  # Centre our map on the region of interest
Map

In [ ]:
# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
sample_region = point.buffer(sample_radius)     # Buffer the point

# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

# Filter to desired region and date bounds
landsat8_collection = landsat8_collection.filterBounds(sample_region)
landsat8_collection = landsat8_collection.sort('system:time_start')  # Sort by image date
landsat8_collection = landsat8_collection.filter(ee.Filter.lt('CLOUD_COVER', 70))

# Filter cloudy pixels within the images
landsat8_collection = landsat8_collection.map(
    lambda image: image.updateMask(ee.Algorithms.Landsat.simpleCloudScore(image).select(['cloud']).lte(20))
)

landsat8_collection

## Time-series mean seasonal NDI

In [ ]:

# Create an empty table with the relevant columns ('year' and 'ndi')
seasonal_ndi_record = pd.DataFrame({'year': [], type_of_ndi: []})

# Simple 'loading bar'
print('Year')

# Loop through years, calculating the mean NDI within the region of interest

for year in list(range(year_begin, year_end+1)): # For every year in the range year_begin to year_end...
    print(f'{year}... ', end='')  # Another 'loading bar' component

    # Filter images only to those within the chosen month and year
    month_images = landsat8_collection.filter(ee.Filter.calendarRange(month_begin, month_end, 'month')).filter(ee.Filter.calendarRange(year, year, 'year'))

    # If there are no available images, return 'NaN' ('not a number'; blank).
    if month_images.size().getInfo() == 0:
        mean_ndi = None

    # Else (i.e. if there *are* images...)
    else:
        # Calculate NDI for every image in the image collection (this is called 'mapping')
        month_ndis = month_images.map(lambda image: image.normalizedDifference([r_high, r_low]).rename(type_of_ndi))

        # Calculate a composite NDI image by calculating the median through the image collection
        month_ndi = month_ndis.median()

        # Calculate the average NDI of the composite image within the region of interest
        mean_ndi = month_ndi.reduceRegion(reducer=ee.Reducer.mean(), geometry=sample_region, scale=30).getInfo()[type_of_ndi]

    df = pd.DataFrame({'year': [year], type_of_ndi: [mean_ndi]})
    seasonal_ndi_record = pd.concat([seasonal_ndi_record, df])

print('Finished!')

In [ ]:
seasonal_ndi_record

In [ ]:
# create an empty figure
fig, ax = plt.subplots(layout='constrained')

# plot the NDI record from the pandas DataFrame
seasonal_ndi_record.plot('year', type_of_ndi, ax=ax)

# Set the x and y labels
ax.set_xlabel('Year')
ax.set_ylabel(type_of_ndi)

## Export data as `.csv` files to Google Drive

In [ ]:
# Construct the filename automatically
filename = location_name + '_seasonal_ndi.csv'

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '\n')

# Export the pandas dataframe to a csv file
seasonal_ndi_record.to_csv(f'/content/drive/My Drive/{folder}/{filename}', index=False)

print('Saved.')